In [1]:
#!pip install optuna
#!pip install flask



# Hyperparameter Tuning

## Using Optuna

In [3]:
import optuna
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2)

def objective(trial):   

    n_estimators = trial.suggest_int('n_estimators', 2, 100)
    max_depth = trial.suggest_int('max_depth', 1, 32)
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)

    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy


c:\Users\hemam\miniconda3\envs\mlops\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Best trial:')
trial = study.best_trial
print('  Value: {}'.format(trial.value))
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))


[I 2025-01-07 21:51:37,764] A new study created in memory with name: no-name-62bf46b9-0766-4f8c-afea-c341bc6bcd47
[I 2025-01-07 21:51:37,815] Trial 0 finished with value: 1.0 and parameters: {'n_estimators': 53, 'max_depth': 12}. Best is trial 0 with value: 1.0.
[I 2025-01-07 21:51:37,852] Trial 1 finished with value: 1.0 and parameters: {'n_estimators': 38, 'max_depth': 32}. Best is trial 0 with value: 1.0.
[I 2025-01-07 21:51:37,871] Trial 2 finished with value: 0.9666666666666667 and parameters: {'n_estimators': 21, 'max_depth': 13}. Best is trial 0 with value: 1.0.
[I 2025-01-07 21:51:37,967] Trial 3 finished with value: 1.0 and parameters: {'n_estimators': 98, 'max_depth': 5}. Best is trial 0 with value: 1.0.
[I 2025-01-07 21:51:38,001] Trial 4 finished with value: 1.0 and parameters: {'n_estimators': 32, 'max_depth': 24}. Best is trial 0 with value: 1.0.
[I 2025-01-07 21:51:38,036] Trial 5 finished with value: 1.0 and parameters: {'n_estimators': 50, 'max_depth': 28}. Best is tri

Best trial:
  Value: 1.0
  Params: 
    n_estimators: 53
    max_depth: 12


## Using Scikit-learn’s GridSearchCV

In [6]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20, 30]
}


In [8]:
clf = RandomForestClassifier()
grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)


grid_search.predict(X_test)


Best parameters found:  {'max_depth': None, 'n_estimators': 50}


array([1, 2, 2, 1, 2, 0, 2, 1, 2, 2, 2, 0, 0, 0, 0, 0, 2, 1, 0, 1, 1, 1,
       2, 0, 1, 1, 0, 1, 1, 0])

In [6]:
import joblib

# Save the model
joblib.dump(clf, 'model.pkl')


['model.pkl']

# Model Packaging

## Create a Flask Application

In [7]:
from flask import Flask, request, jsonify
import joblib

app = Flask(__name__)
model = joblib.load('model.pkl')  # Ensure this path is correct

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    prediction = model.predict([data['input']])
    return jsonify({'prediction': prediction.tolist()})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.4:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Jan/2025 21:24:58] "GET / HTTP/1.1" 404 -


## Create a Dockerfile

In [8]:
FROM python:3.8-slim

WORKDIR /app

COPY requirements.txt requirements.txt
RUN pip install -r requirements.txt

COPY . .

CMD ["python", "app.py"]


SyntaxError: invalid syntax (1181971237.py, line 1)

## Build and Run Docker Container

In [2]:
docker build -t my_model .
docker run -p 5000:5000 my_model


SyntaxError: invalid syntax (1328916582.py, line 1)